# Let's build a generative pretrained transformer, from scratch!

Transformer: a type of neural net architecture.\
Dataset: shakespeare dataset


#### SETUP

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [1]:
import urllib.request
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
filename = 'shakespeare.txt'
urllib.request.urlretrieve(url, filename)

('shakespeare.txt', <http.client.HTTPMessage at 0x7f81d8f24eb0>)

In [2]:
with open('./shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
# all unique chars that occur in this dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f"Length: {vocab_size}")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Length: 65


In [4]:
# tokenize the chars into ints by creating a lookup table; many other schemas for doing this
# google uses SentencePiece, and OpenAI tiktoken
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode('Hi!'))
print(decode(encode('Hi!')))

[20, 47, 2]
Hi!


In [11]:
# encode the entire text dataset and store it inside of a tensor
data = torch.tensor(encode(text), dtype=torch.long)

In [6]:
# finally, let's separate out the data into a train and validation split

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
# for all of these blocks of data, we also have mini-batches for efficiency reasons
# these keep the GPUs busy, as they are very good for parallel processing of data that
# can process these in parallel

torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # get random positions to get blocks from
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} the target is: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
When input is [24] the target is: 43
When input is [24, 43] the target is: 58
When input is [24, 43, 58] the target is: 5
When input is [24, 43, 58, 5] the target is: 57
When input is [24, 43, 58, 5, 57] the target is: 1
When input is [24, 43, 58, 5, 57, 1] the target is: 46
When input is [24, 43, 58, 5, 57, 1, 46] the target is: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the target is: 39
When input is [44] the target is: 53
When input is [44, 53] the target is: 56
When input is [44, 53, 56] the target is: 1
When input is [44, 53, 56, 1] the target is: 58
When input is [44, 53, 56, 1, 58] 

#### Let's now insert a single self-attention block to our network

In [13]:
# scale up the model
batch_size = 64 # independent sequences to proces in parallel
block_size = 256 # maximum content length for predictions, up from 'eight
n_head = 5
n_embd = 65
dropout = 0.2 # 20% of intermediate calculations are dropped out

In [14]:
class Head(nn.Module): # single head of self-attention
    def __init__(self, head_size, n_embd):        
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [20]:
# communication
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size, n_embd):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, n_embd) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd) # residual connection projection back to residual pathway
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
# TODO EXERCISE ONE: combining Head and MultiHeadAttention into one class that processes all of the heads in parallel, treating the
# heads as another batch dimension. In other words, 
# Right now, MultiHeadAttention is processing each head in sequence, which is not efficient. The steps:
# 1. Take in all of the heads

class MultiHeadAttentionParallel(nn.Module):
    def __init__(self, num_heads):
        super().__init__()

In [17]:
# computation, interspersed through the network. All of the tokens independently compute
# note in this paper: https://arxiv.org/abs/1512.03385, authors mention
# that while dmodel = 512, the inner-layer has dff = 2048. Different parameters are used
# layer to layer. Therefore, we should actually multiply the dmodel by 4 for the inner layer
# this is a typical transformer design pattern

class FeedForward(nn.Module):
    # simple linear layer followed by a nonlinearity (ie. 0s for negatives)

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            # I can also put nn.Linear(n_embd, n_embd) here instead of self.proj
        )
        self.proj = nn.Linear(4*n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = self.net(x)
        out = self.dropout(self.proj(out))
        return out

In [18]:
# transformer: communication -> computation -> communication -> computation -> ...
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size, n_embd)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # adding in 'x +' for residual connections, the sa is the forked off computation
        x = x + self.ffwd(self.ln2(x))
        return x

I have some overfitting in training (as seen in the diff between training and val loss) so I will also include some dropout.

In [19]:
torch.manual_seed(1337)

class GPTLanguageModel(nn.Module):
    def __init__(self, n_embd): # head_size is n_embd (dimensionality of model), but divided not by # heads
        super().__init__()
        # each token directly reads off logits for the next token, from the lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        
        # add a positional encoding
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # self.sa_heads = MultiHeadAttention(6, n_embd // 6)
        # self.ffwd = FeedForward(n_embd)

        self.blocks = nn.Sequential(
            Block(n_embd, n_head),
            Block(n_embd, n_head),
            Block(n_embd, n_head),
            nn.LayerNorm(n_embd) # also add this right before the end of the transformer
        )

        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        # idx and target are both (B, T) tensor of integers
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C), holding both token identities and positions. Doesn't matter much in a Bigram model where history is only 1 step
        # x = self.sa_heads(x) 
        # # Feedforward gives tokens time to *think* before making a prediction via logits
        # x = self.ffwd(x) # multiplied by 4?
        x = self.blocks(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            # Pytorch takes in Channel as the second dimension, so we need to reshape
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) # This will shape into a single dimension, data is maintained but shape changes
            loss = F.cross_entropy(logits, targets) # Measures quality of the logits wrt the targets

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # make sure we don't accidentally get an idx outside of block size
            idx_cond = idx[:, -block_size:] # (B, T)

            # get predictions
            logits, loss = self(idx_cond) # calls the forward function, without targets (no ground truth to compare against, just generating)
            # loss not used but we still pluck it out to make sure logits isn't a tuple containing both

            # focus only on the last time step
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1)

            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

    
m2 = GPTLanguageModel(n_embd)
logits, loss = m2(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.2362, grad_fn=<NllLossBackward>)


In [260]:
# optimizer2 = torch.optim.AdamW(m2.parameters(), lr=1e-3)
optimizer2 = torch.optim.AdamW(m2.parameters(), lr=3e-4) # reducing learning rate after scaling up the model

In [263]:
max_iters = 5000
eval_interval = 100
eval_iters = 200

In [ ]:
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        out = {}
        m2.eval()
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = get_batch(split)
                logits, loss = m2(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
        m2.train()
        print(f"step {iter}: train loss {out['train']:.4f}, val loss {out['val']:.4f}")

    # sample a bunch of data
    xb, yb = get_batch('train') # function we defined earlier gets a random batch of data from the training set

    # evaluate the loss for bigram model we defined earlier and can call
    logits, loss = m2(xb, yb)
    optimizer2.zero_grad(set_to_none=True)
    loss.backward()
    optimizer2.step()

In [ ]:
idx = torch.zeros((1, 1), dtype=torch.long) # 1x1 tensor holding a zero, starting with uniline
output = m2.generate(idx, max_new_tokens=500)[0].tolist() # [0] just makes sure it isn't a nested tensor, and tolist() converts the tensor to an array
print(decode(output)) # we defined an int to char mapping earlier. This will be awful as we haven't trained the model yet

Find the GPU-version here: https://colab.research.google.com/drive/1rI22Stc8px3af_atuK8a17Zi2ahaMkEB